<a href="https://colab.research.google.com/github/irinap9270/hg_ml/blob/main/Ch5-3.%20EnsembleLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# """랜덤 포레스트"""

> 부트스트랩 샘플

: 입력한 훈련 데이터에서 중복을 허용하여 랜덤하게 샘플을 추출

: 기본적으로 훈련 세트의 크기와 동일

> RandomForestClassifier

: 전체 특성 개수의 제곱근만큼 특성 선택

: 각 트리의 클래스별 확률 평가 > 가장 높은 확률을 가진 클래스 선택

In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

# 교차 검증 수행
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

# oob_score; oob샘플로 평가한 점수
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

# 랜덤 포레스트 모델을 훈련 세트에 훈련한 후 특성 중요도 출력
rf.fit(train_input, train_target)
print(rf.feature_importances_)
print(rf.oob_score_)

0.9973541965122431 0.8905151032797809
[0.23167441 0.50039841 0.26792718]
0.8934000384837406


# """엑스트라 트리"""

> 전체 훈련 세트

: 부트스트랩 샘플을 사용하지 않음

: 무작위로 분할

In [7]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
et.fit(train_input, train_target)
print(et.feature_importances_)

0.9974503966084433 0.8887848893166506
[0.20183568 0.52242907 0.27573525]


# """그레이디언트 부스팅"""

> 깊이가 얕은 결정 트리

: 이전 트리의 오차 보완

: 과대적합에 강함

: 높은 일반화 성능

> 경사 하강법을 사용

: 결정 트리를 계속 추가하면서 가장 낮은 곳을 찾아 이동

In [10]:
from sklearn.ensemble import GradientBoostingClassifier
# n_estimators; 결정 트리 개수
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
gb.fit(train_input, train_target)
print(gb.feature_importances_)

0.9464595437171814 0.8780082549788999
[0.15872278 0.68010884 0.16116839]


# """히스토그램 기반 그레이디언트 부스팅"""

> 입력 특성을 256 구간으로 나눔

: 노드 분할에서 최적의 분할을 빠르게 찾을 수 있음

: 입력에 누락된 특성이 있어도 전처리 필요 없음

> HistGradientBoostingClassifier

: 기본 매개변수에서 안정적인 성능

: 트리 개수 지정 대신에 max_iter를 사용하여 부스팅 반복 횟수 지정

In [12]:
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

from sklearn.inspection import permutation_importance # 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지 관찰하는 함수
hgb.fit(train_input, train_target)
# 훈련 세트에서 특성 중요도 계산
# n_repeats; 랜덤하게 섞을 횟수 지정
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

# 테스트 세트에서 특성 중요도 계산
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

# 테스트 세트에서의 성능 최종 확인
hgb.score(test_input, test_target)

0.9321723946453317 0.8801241948619236
[0.08876275 0.23438522 0.08027708]
[0.05969231 0.20238462 0.049     ]


0.8723076923076923

XGBoost를 활용하여 그레이디언트 부스팅 알고리즘을 사용해보자.

In [13]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9558403027491312 0.8782000074035686


 LightGBM 을 활용하여 그레이디언트 부스팅 알고리즘을 사용해보자.

In [14]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

[LightGBM] [Info] Number of positive: 3151, number of negative: 1006
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000448 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 372
[LightGBM] [Info] Number of data points in the train set: 4157, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.757999 -> initscore=1.141738
[LightGBM] [Info] Start training from score 1.141738
[LightGBM] [Info] Number of positive: 3151, number of negative: 1006
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000258 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 370
[LightGBM] [Info] Number of data points in the train set: 4157, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.757999 -> initscore=1.141738
[LightGBM] [Info] Start training from score 1.141738
[LightGBM] [Info] Number o